In [ ]:
# remember to create and activate the data_env environment first (or install specified libraries in base environment)
# also, run preprocess_data.ipynb first
import os
import json
from collections import defaultdict
import pandas as pd
import numpy as np
import pubchempy as pcp
import deepchem as dc
from rdkit import Chem
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error

In [ ]:
def concordance_correlation_coefficient(y_true, y_pred):
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)
    std_true = np.std(y_true)
    std_pred = np.std(y_pred)
    pearson_correlation_coefficient = pearsonr(y_true, y_pred)[0]
    ccc = (2*pearson_correlation_coefficient*std_true*std_pred)/(std_true**2 + std_pred**2 + (mean_true - mean_pred)**2)
    return ccc

# Correlation of duplicated cell-line–drug experiments in GDSC2v8.5

In [ ]:
# either: https://www.cancerrxgene.org/downloads/genetic_features?screening_set=GDSC2&tissue=PANCANCER&mutation=both
# or: https://www.cancerrxgene.org/downloads/bulk_download -> GDSC2-dataset (almost the same, but has a few less measurements)
pancancer_gdsc2_raw = pd.read_csv('GDSC_Drug_Data/PANCANCER_IC_GDSC2.csv')

In [ ]:
pancancer_gdsc2_raw_duplicate_mask = pancancer_gdsc2_raw.duplicated(subset=['Cell Line Name', 'Drug Name'], keep=False)

In [ ]:
pancancer_gdsc2_raw_ic50 = pancancer_gdsc2_raw[pancancer_gdsc2_raw_duplicate_mask].sort_values(by=['Cell Line Name', 'Drug Name'])[['Cell Line Name', 'Drug Name', 'IC50']]
pancancer_gdsc2_raw_auc = pancancer_gdsc2_raw[pancancer_gdsc2_raw_duplicate_mask].sort_values(by=['Cell Line Name', 'Drug Name'])[['Cell Line Name', 'Drug Name', 'AUC']]

In [ ]:
len(pancancer_gdsc2_raw_ic50)/2, len(set(pancancer_gdsc2_raw_ic50['Cell Line Name']))

In [ ]:
duplicate_drugs_ic50_dict = dict()
for dn in set(pancancer_gdsc2_raw_ic50['Drug Name']):
    duplicate_drugs_ic50_dict[dn] = []
for x in pancancer_gdsc2_raw[pancancer_gdsc2_raw_duplicate_mask].groupby(['Drug ID', 'Drug Name']):
    duplicate_drugs_ic50_dict[x[0][1]].append(list(x[1].sort_values(by='Cell Line Name')['IC50']))

In [ ]:
cccs = []
for k, v in duplicate_drugs_ic50_dict.items():
    p = pancancer_gdsc2_raw_ic50[pancancer_gdsc2_raw_ic50['Drug Name'] == k]['IC50']
    print(k, concordance_correlation_coefficient(v[0], v[1]), mean_absolute_error(v[0], v[1]), np.std(p))
    cccs.append(concordance_correlation_coefficient(v[0], v[1]))

In [ ]:
np.mean(cccs), np.std(cccs)

In [ ]:
duplicate_drugs_auc_dict = dict()
for dn in set(pancancer_gdsc2_raw_auc['Drug Name']):
    duplicate_drugs_auc_dict[dn] = []
for x in pancancer_gdsc2_raw[pancancer_gdsc2_raw_duplicate_mask].groupby(['Drug ID', 'Drug Name']):
    duplicate_drugs_auc_dict[x[0][1]].append(list(x[1].sort_values(by='Cell Line Name')['AUC']))

In [ ]:
cccs = []
for k, v in duplicate_drugs_auc_dict.items():
    p = pancancer_gdsc2_raw_auc[pancancer_gdsc2_raw_auc['Drug Name'] == k]['AUC']
    print(k, concordance_correlation_coefficient(v[0], v[1]), mean_absolute_error(v[0], v[1]), np.std(p))
    cccs.append(concordance_correlation_coefficient(v[0], v[1]))

In [ ]:
np.mean(cccs), np.std(cccs)

# Comparison between CCLE (from TGSA paper) and GDSC2v8.5 features (was not needed and thus not mentioned in our paper)

In [ ]:
# https://cellmodelpassports.sanger.ac.uk/downloads -> Gene Annotation -> under Gene List, click View all versions
gene_annot = pd.read_csv('Bulk_Cell_line_Genomic_Data/Cell_Model_Passports/Gene_Annotation/gene_identifiers_20191101.csv', dtype=object)

In [ ]:
id2gene_dict = dict(zip(gene_annot['entrez_id'].str.split('.').str[0].map(lambda x: '({})'.format(x)), gene_annot['hgnc_symbol']))

In [ ]:
# file from TGSA paper
cell_annot = pd.read_csv('TGSA_Data/PANCANCER_IC_82833_580_170.csv')
id2cell_dict = dict(zip(cell_annot['DepMap_ID'], cell_annot['Cell line name']))

In [ ]:
# files from TGSA paper
mu = pd.read_csv('TGSA_Data/mu.csv', index_col=0).sort_index()
cn = pd.read_csv('TGSA_Data/cn.csv', index_col=0).sort_index()
ex = pd.read_csv('TGSA_Data/exp.csv', index_col=0).sort_index()

In [ ]:
mu = mu.rename(index=id2cell_dict, columns=id2gene_dict)
cn = cn.rename(index=id2cell_dict, columns=id2gene_dict)
ex = ex.rename(index=id2cell_dict, columns=id2gene_dict)

In [ ]:
# to get only the 580 cell lines and 706 genes used in TGSA paper (cn or exp file would have also been possible instead of mu)
cell_list = list(mu.index)
gene_list = list(mu.columns)

In [ ]:
# to get these datasets, run preprocess_data.ipynb
mut = pd.read_csv('../features/cell_features/mut_all.csv', index_col=0)
cnv = pd.read_csv('../features/cell_features/cnv_all.csv', index_col=0)
exp = pd.read_csv('../features/cell_features/exp_tpm.csv', index_col=0)

In [ ]:
mut_renamed = mut.copy()
cnv_renamed = cnv.copy()
exp_renamed = exp.copy()

In [ ]:
mut_renamed.columns = [i.split('.')[0] for i in mut_renamed.columns]
cnv_renamed.columns = [i.split('.')[0] for i in cnv_renamed.columns]
exp_renamed.columns = [i.split('.')[0] for i in exp_renamed.columns]

In [ ]:
# the following three notebook cells could have been solved with pandas reindex, not needing the cell_list and gene_list
cell_list_mut = list(set(cell_list) - set(['GMS-10', 'HD-MY-Z', 'LS-1034', '42-MG-BA', 'SNU-182', 'VM-CUB-1', 'NCI-H1876', 'SW900', 'CL-34', 'EGI-1'])) # cell lines are in TGSA mu dataset, but not in GDSC2v8.5 mut_all dataset (can be found out by commenting this line out)
gene_list_mut = list(set(gene_list) - set(['SEPT9', 'FGFR1OP', 'H3F3A', 'SEPT6', 'MKL1', 'BIVM-ERCC5', 'FAM46C', 'HIST1H4I', 'CARS', 'SEPT5', 'H3F3B'])) # genes are in TGSA mu dataset, but not in GDSC2v8.5 mut_all dataset (can be found out by commenting this line out)
mut_only_tgsa_overlaps = mut_renamed.loc[cell_list_mut, gene_list_mut]

In [ ]:
cell_list_cnv = list(set(cell_list) - set(['697', 'NCI-H2052', 'LS-1034', 'WSU-DLCL2', 'EBC-1', '5637', 'ES-2', 'DND-41', 'Jurkat'])) # cell lines are in TGSA cn dataset, but not in GDSC2v8.5 cnv_all dataset (can be found out by commenting this line out)
gene_list_cnv = list(set(gene_list) - set(['DCAF12L2', 'FOXO4', 'STAG2', 'PHF6', 'AMER1', 'MED12', 'DDX3X', 'BCOR', 'ELF4', 'RBM10', 'SEPT6', 'AR', 'TFE3', 'GPC3', 'SSX1', 'NONO', 'BCORL1', 'IRS4', 'ATRX', 'FAM47C', 'GATA1', 'ARAF', 'KDM6A', 'SMC1A', 'RPL10', 'KDM5C', 'CRLF2', 'ZRSR2', 'FLNA', 'C15orf65', 'BTK', 'WAS', 'MSN', 'P2RY8', 'ZMYM3', 'NUTM2B', 'ATP2B3', 'MTCP1'])) # genes are in TGSA cn dataset, but not in GDSC2v8.5 cnv_all dataset (can be found out by commenting this line out)
cnv_only_tgsa_overlaps = cnv_renamed.loc[cell_list_cnv, gene_list_cnv]

In [ ]:
cell_list_exp = list(set(cell_list) - set(['LS-1034'])) # cell lines are in TGSA exp dataset, but not in GDSC2v8.5 ge_readcount dataset (can be found out by commenting this line out)
gene_list_exp = list(set(gene_list) - set(['SEPT9', 'FGFR1OP', 'SEPT6', 'H3F3A', 'H3F3B', 'SEPT5', 'FAM46C', 'MKL1', 'HIST1H4I', 'CARS'])) # genes are in TGSA exp dataset, but not in GDSC2v8.5 ge_readcount dataset (can be found out by commenting this line out)
exp_only_tgsa_overlaps = exp_renamed.loc[cell_list_exp, gene_list_exp]

In [ ]:
mu = mu.loc[cell_list_mut, gene_list_mut]
cn = cn.loc[cell_list_cnv, gene_list_cnv]
ex = ex.loc[cell_list_exp, gene_list_exp]

In [ ]:
a = mut_only_tgsa_overlaps.to_numpy().flatten()
b = mu.to_numpy().flatten()
concordance_correlation_coefficient(a, b), np.logical_and((a == 1), (b == 1)).sum()/np.logical_or((a == 1), (b == 1)).sum()
# only 22.25% of ones overlap, pretty uncomparable

In [ ]:
a = cnv_only_tgsa_overlaps.to_numpy().flatten()
b = cn.to_numpy().flatten()
pearsonr(a, b)[0] # another measure does not make much sense because values of cn are not in {-2, -1, 0, 1, 2} unlike cnv_only_tgsa_overlaps

In [ ]:
a = exp_only_tgsa_overlaps.to_numpy().flatten()
b = ex.to_numpy().flatten()
concordance_correlation_coefficient(a, b) # very comparable

# Comparison between CCLE (24Q2) and GDSC2v8.5 input data

In [ ]:
gdsc2_ic_shared_with_ccle = pd.read_csv('../targets/gdsc2_ic_shared_with_ccle.csv', index_col=0)
gdsc2_mut_shared_with_ccle = pd.read_csv('../features/cell_features/gdsc2_mut_shared_with_ccle.csv', index_col=0)
gdsc2_cnv_shared_with_ccle = pd.read_csv('../features/cell_features/gdsc2_cnv_shared_with_ccle.csv', index_col=0)
gdsc2_exp_shared_with_ccle = pd.read_csv('../features/cell_features/gdsc2_exp_shared_with_ccle.csv', index_col=0)

ccle_ic_shared_with_gdsc2 = pd.read_csv('../targets/ccle_ic_shared_with_gdsc2.csv', index_col=0)
ccle_mut_shared_with_gdsc2 = pd.read_csv('../features/cell_features/ccle_mut_shared_with_gdsc2.csv', index_col=0, low_memory=False)
ccle_cnv_shared_with_gdsc2 = pd.read_csv('../features/cell_features/ccle_cnv_shared_with_gdsc2.csv', index_col=0)
ccle_exp_shared_with_gdsc2 = pd.read_csv('../features/cell_features/ccle_exp_shared_with_gdsc2.csv', index_col=0)

gdsc2_ic_capped_shared_with_ccle = pd.read_csv('../targets/gdsc2_ic_capped_shared_with_ccle.csv', index_col=0)

In [ ]:
a = gdsc2_ic_shared_with_ccle.to_numpy().flatten()
b = ccle_ic_shared_with_gdsc2.to_numpy().flatten()
a, b = a[a == a], b[a == a]
a, b = a[b == b], b[b == b]
concordance_correlation_coefficient(a, b)
# CCLE and GDSC IC50s are kind of comparable, but not really
# differences of screening:
# CCLE doses are .0025, .0080, .025, .080, .25, .80, 2.53, 8 (rounded values, the second-last one is actually 2.531646; sometimes not the whole range are used), IC50 is capped at maximum tested dose (either 2.53 or 8)
# GDSC2 uses not only two (as in CCLE only 2.53 and 8 are used as maximum dose), but a lot of maximum doses: .01, .0125, .02, .1, .2, .25, .5, .8, 1, 2, 2.5, 3, 4, 5, 8, 10, 20, 30, 32, 40, 50, 100, 121, 640, 2000, IC50 is not capped at maximum tested dose

In [ ]:
# we capped the IC50 in GDSC to the maximum tested dose
# almost no difference to uncapped dataset, thus we did not mention it in our paper
a = gdsc2_ic_capped_shared_with_ccle.to_numpy().flatten()
b = ccle_ic_shared_with_gdsc2.to_numpy().flatten()
a, b = a[a == a], b[a == a]
a, b = a[b == b], b[b == b]
concordance_correlation_coefficient(a, b)

In [ ]:
a = gdsc2_mut_shared_with_ccle.to_numpy().flatten()
b = ccle_mut_shared_with_gdsc2.to_numpy().flatten()
concordance_correlation_coefficient(a, b), np.logical_and((a == 1), (b == 1)).sum()/np.logical_or((a == 1), (b == 1)).sum()
# only 11.65% of ones overlap, pretty uncomparable

In [ ]:
a = gdsc2_cnv_shared_with_ccle.to_numpy().flatten()
b = ccle_cnv_shared_with_gdsc2.to_numpy().flatten()
pearsonr(a, b)[0] # another measure does not make much sense because values of cn are not in {-2, -1, 0, 1, 2} unlike ccle_cnv_shared_with_gdsc2

In [ ]:
a = gdsc2_exp_shared_with_ccle.to_numpy().flatten()
b = ccle_exp_shared_with_gdsc2.to_numpy().flatten()
concordance_correlation_coefficient(a, b) # very comparable

# GDSC2: Do PAN-CANCER MUT columns equal the corresponding MUT columns?

In [ ]:
pancancer_genetic_features_gdsc2 = pd.read_csv('../features/cell_features/pancancer_genetic_features_gdsc2.csv', index_col=0)

In [ ]:
pancancer_genetic_features_gdsc2.columns = [i.replace('_', '.') for i in pancancer_genetic_features_gdsc2.columns]

In [ ]:
num_genes_ending_with_mut = 0
for i in pancancer_genetic_features_gdsc2.columns:
    if i.endswith('.mut'):
        num_genes_ending_with_mut += 1
num_genes_ending_with_mut

In [ ]:
shared_rows_pancan_mut = list(set(pancancer_genetic_features_gdsc2.index).intersection(set(mut.index)))
shared_cols_pancan_mut = list(set(pancancer_genetic_features_gdsc2.columns).intersection(set(mut.columns)))
len(shared_rows_pancan_mut), len(shared_cols_pancan_mut) # 920 out of 925 cell lines and 285 out of 310 mut features (310 from cell above)

In [ ]:
a = pancancer_genetic_features_gdsc2.loc[shared_rows_pancan_mut, shared_cols_pancan_mut].to_numpy().flatten()
b = mut.loc[shared_rows_pancan_mut, shared_cols_pancan_mut].to_numpy().flatten()
concordance_correlation_coefficient(a, b), np.logical_and((a == 1), (b == 1)).sum()/np.logical_or((a == 1), (b == 1)).sum()
# pan-cancer mut features differ a lot from mut_all features, only 6% of ones overlap